In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# Torch Tensors

### Tensors are n-dimenstional arrangements of object
Vector is first order tensor, matrix is a second order tensor.

### Vectors

$$
\mathbf{y}=
\begin{bmatrix}
9\\
4\\
24\\
-12
\end{bmatrix}
$$


In [ ]:
y = torch.tensor([9,4,24,-12]).float()
y

tensor([  9.,   4.,  24., -12.])

### Multidimensional tensor

$$
A=
\begin{bmatrix}
1 & 3 & 2 & -1\\
5 & 2 & 1 & -2\\
0 & 1 & 2 & 4\\
1 & 1 & -1 & -3
\end{bmatrix}
$$

In [ ]:
A = torch.tensor([[1,3,2,-1],[5,2,1,-2],[0,1,2,4],[1,1,-1,-3]]).float()
A

tensor([[ 1.,  3.,  2., -1.],
        [ 5.,  2.,  1., -2.],
        [ 0.,  1.,  2.,  4.],
        [ 1.,  1., -1., -3.]])

### Numpy arrays to torch tensors

In [ ]:
B = np.array([[1,0],[0,1]])
torch.tensor(B).float()

tensor([[1., 0.],
        [0., 1.]])

# Building Neural Network models with torch

### General Structure

In [ ]:
class Ne(nn.Module):
    # Intialise your trainable layers here
    def __init__(self):
        super().__init__()

    # Define a forward pass with your trainable and functional layers here
    def forward(self, x):
        pass

### Example Model for Mnist Dataset

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 channel 28 x 28 -> 10 channels 24 x 24, 10 filters, 5 by 5, 250 trainable params
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        # 10 channel 12 x 12 -> 20 channel 8 x 8, 200 filters, 5 by 5, 5000 trainable params
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        # 320 neurons -> 50 neurons, 16000 trainable params + 50
        self.fc1 = nn.Linear(320, 50)
        # 50 neurosn -> 10 neurons, 500 trainable params + 10
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9)

# Dataset loading

https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])
trainset = datasets.MNIST('data', train=True, download=True, transform=transform)
valset = datasets.MNIST('data', train=False, transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=640, shuffle=True)
test_loader = torch.utils.data.DataLoader(valset, batch_size=640, shuffle=True)

# Training

In [ ]:
network.train()
for epoch in range(3):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

<ipython-input-61-6c2218f37724>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 0 [0/60000 (0%)]	Loss: 0.393257
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.363937
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.383027
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.317677
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.385315
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.410912
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.435674
Train Epoch: 0 [44800/60000 (74%)]	Loss: 0.343292
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.345185
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.300280
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.344296
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.353807
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.292317
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.376967
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.419868
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.281213
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.357538
Train Epoch: 1 [44800/60000 (74%)]	Loss: 0.299612
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.342457
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.249130
Train Epoch:

# Validation

In [ ]:
network.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = network(data)
        test_loss += F.nll_loss(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

<ipython-input-61-6c2218f37724>:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 0.0942, Accuracy: 9708/10000 (97%)

